### Import and setup data

We are just importing the json file and printing out the data section ensuring that we can access the data.

In [8]:
import json

with open('ntd-json-data.json', 'r') as file:
    raw = json.load(file)

# Focus on actual data
data = raw["data"]

# Data comes up as an array of arrays, with each array corresponding to a row in the dataset table, e.g... 
print(json.dumps(data[0], indent=2))

[
  "row-jzh9~3jst-vtgf",
  "00000000-0000-0000-ECEB-FE81C09A5D87",
  0,
  1708100603,
  null,
  1708100603,
  null,
  "{ }",
  "MTA New York City Transit",
  "Brooklyn",
  "NY",
  "20008",
  "Subsidiary Unit of a Transit Agency, Reporting Separately",
  "Full Reporter",
  "63217",
  "New York--Jersey City--Newark, NY--NJ",
  "19426449",
  "10019",
  "CB",
  "DO",
  "443",
  "3697741",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "15739148",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "0",
  null,
  "4.2564",
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null,
  null
]


### Refactor each row from an array into a dictionary

Now that we have the data, we can see that each row in the table is represented as an array in this data. Arrays are a bit ambiguous when it comes to accessing its data, so I think we should convert the arrays to a dictionary so we can access data in a cleaner way. 

Note: As I am changing each row array into a dictionary, I am also getting rid of any keys/column names that have "Questionable" in it. This appears after almost every column that reports some type of numerical data, and I believe it is used if their source is unreliable for that data field. I haven't seen it marked before, and it just clutters our dictionary, so I am going to remove them for the time being.

In [9]:
# Retrieve all column values
keys = [column["name"] for column in raw["meta"]["view"]["columns"]]

# Map column names to each value in the row, and remove "Questionable" fields
converted_data = [{k: v for k, v in dict(zip(keys, row)).items() if "Questionable" not in k} for row in data]

# Now each element in the array is a dictionary with mapped keys to each value
print(json.dumps(converted_data[0], indent=2))

{
  "sid": "row-jzh9~3jst-vtgf",
  "id": "00000000-0000-0000-ECEB-FE81C09A5D87",
  "position": 0,
  "created_at": 1708100603,
  "created_meta": null,
  "updated_at": 1708100603,
  "updated_meta": null,
  "meta": "{ }",
  "Agency": "MTA New York City Transit",
  "City": "Brooklyn",
  "State": "NY",
  "NTD ID": "20008",
  "Organization Type": "Subsidiary Unit of a Transit Agency, Reporting Separately",
  "Reporter Type": "Full Reporter",
  "UACE Code": "63217",
  "UZA Name": "New York--Jersey City--Newark, NY--NJ",
  "Primary UZA Population": "19426449",
  "Agency VOMS": "10019",
  "Mode": "CB",
  "TOS": "DO",
  "Mode VOMS": "443",
  "Diesel (gal)": "3697741",
  "Gasoline (gal)": "0",
  "Liquefied Petroleum Gas (gal equivalent)": "0",
  "Compressed Natural Gas (gal equivalent)": "0",
  "Bio-Diesel (gal)": "0",
  "Hydrogen (kg)": "0",
  "Other Fuel (gal/gal equivalent)": "0",
  "Electric Propulsion (kWh)": "0",
  "Electric Battery (kWh)": "0",
  "Diesel (miles)": "15739148",
  "Gasoline (

### Aggregate all rows with same UACE code into one area

Now that each row is turned into a dictionary, lets organize the data by UACE code. The code will be the key in a dictionary which will hold an array of entries that all pertain to that code. 

In [14]:
aggregate_data = {}

for row in converted_data:
    code = row["UACE Code"]
    if aggregate_data.get(code) == None:
        aggregate_data[code] = [row]
    else:
        aggregate_data[code].append(row)

# Show first few rows of data in 63217 area code
print(json.dumps(aggregate_data["63217"][0:3], indent=2))
print("Number of entries for 63217: " + str(len(aggregate_data["63217"])))

[
  {
    "sid": "row-jzh9~3jst-vtgf",
    "id": "00000000-0000-0000-ECEB-FE81C09A5D87",
    "position": 0,
    "created_at": 1708100603,
    "created_meta": null,
    "updated_at": 1708100603,
    "updated_meta": null,
    "meta": "{ }",
    "Agency": "MTA New York City Transit",
    "City": "Brooklyn",
    "State": "NY",
    "NTD ID": "20008",
    "Organization Type": "Subsidiary Unit of a Transit Agency, Reporting Separately",
    "Reporter Type": "Full Reporter",
    "UACE Code": "63217",
    "UZA Name": "New York--Jersey City--Newark, NY--NJ",
    "Primary UZA Population": "19426449",
    "Agency VOMS": "10019",
    "Mode": "CB",
    "TOS": "DO",
    "Mode VOMS": "443",
    "Diesel (gal)": "3697741",
    "Gasoline (gal)": "0",
    "Liquefied Petroleum Gas (gal equivalent)": "0",
    "Compressed Natural Gas (gal equivalent)": "0",
    "Bio-Diesel (gal)": "0",
    "Hydrogen (kg)": "0",
    "Other Fuel (gal/gal equivalent)": "0",
    "Electric Propulsion (kWh)": "0",
    "Electric Ba

### Write JSON to file

Integrate our revised data into the old data, and then export the file for further use elsewhere.

In [11]:
# Replace the "data" field for our new aggregate_data variable
json_data = raw
json_data["data"] = aggregate_data
json_data = json.dumps(json_data, indent=4)

# Write JSON to file
with open("revised-ntd-json-data.json", 'w') as file:
    file.write(json_data)

### Data manipulation

These are some examples of what we can do with the newly formatted data. 

In [12]:
# Total "fuel + (unit)" count based off area code and mode
def total_count(code, fuel, unit, mode=""):
    total_miles = 0
    total_entries = 0
    for row in aggregate_data[code]:
        if mode == "" or row["Mode"] == mode:
            total_miles += float(row[fuel + " (" + unit + ")"])
            total_entries += 1
    return (total_miles, total_entries)

# Average miles in area code based on fuel type and mode
def average_miles(code, fuel, mode=""):
    (total, entries) = total_count(code, fuel, "miles", mode)
    avg = total / entries if entries > 0 else 0
    print("Area Code: " + code + "\n" + "Total " + fuel + " Miles: " + str(total) + "\nTotal Entries: " + str(entries) + "\nAvg: " + str(avg) + "\n")

# Aggregate and rank fuel mileage based on area code and mode
def ranked_fuel_mileage(code, mode=""):
    fuel_types = [k.split(" (").pop(0) for k in keys if k.endswith("(miles)")]
    aggregate_fuel_mileage = []
    for fuel in fuel_types:
        (total, _) = total_count(code, fuel, "miles", mode)
        aggregate_fuel_mileage.append((fuel + " (miles)", total))
    sorted_aggregate_fuel_mileage = sorted(aggregate_fuel_mileage, key=lambda x: x[1], reverse=True)
    print("Aggregate fuel mileage for UACE " + code + " ranked in descending order...")
    print(sorted_aggregate_fuel_mileage.__str__() + "\n")

### Running data manipulations

In [13]:
average_miles("63217", "Gasoline")

ranked_fuel_mileage("63217")

Area Code: 63217
Total Gasoline Miles: 66437808.0
Total Entries: 58
Avg: 1145479.448275862

Aggregate fuel mileage for UACE 63217 ranked in descending order...
[('Electric Propulsion (miles)', 497061246.0), ('Diesel (miles)', 259872221.0), ('Gasoline (miles)', 66437808.0), ('Compressed Natural Gas (miles)', 32531976.0), ('Other Fuel (miles)', 5410262.0), ('Electric Battery (miles)', 256773.0), ('Liquefied Petroleum Gas (miles)', 0.0), ('Hydrogen (miles)', 0.0)]

